In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,958 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,443 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.8 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,421 k

In [2]:
!pip install htmlmin
!pip install openai
!pip install html2text
!pip install python-dotenv

  Preparing metadata (setup.py) ... done
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=4a6a6eae9bfb3c280a44d359b526f9462aa53ae4f71eb9873aa7c8e02427c4a0
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=5fddac1fad017938c532b30dba3dc1e2f3174ca23666746e0bcb87621c221feb
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd18

In [3]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import requests

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

In [4]:
def get_programs(driver):
    driver.get("https://www.canadavisa.com/canada-pnp-finder-and-tracker.html")
    page_source=""
    try:
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ui celled fixed sortable table"))
        )
    finally:
        # Once the element is loaded, get the page source
        page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    table=soup.find_all("table","ui celled fixed sortable table")[0]
    headers = [th.get_text() for th in table.find('tr').find_all('th')]
    data = []

    for row in table.find_all('tr')[1:]:
        data.append([td.get_text() for td in row.find_all('td')])
    df = pd.DataFrame(data, columns=headers)

    for i, row in enumerate(table.find_all('tr')[1:]):
        row_data = []
        for idx, td in enumerate(row.find_all('td')):
            if idx == 1:
                link = td.find('a')
                if link:
                    df.loc[i, 'Stream Link'] = link['href']
    return df.to_dict(orient='records')


In [5]:
# Define the URL and headers for the request
url = "https://evaluator.canadavisa.com/api/immigration/streams?lang=en"
headers = {
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Origin": "https://www.canadavisa.com",
    "Referer": "https://www.canadavisa.com/canada-pnp-finder-and-tracker.html",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "sec-ch-ua": "\"Not/A)Brand\";v=\"8\", \"Chromium\";v=\"126\", \"Google Chrome\";v=\"126\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\""
}

response = requests.get(url, headers=headers)
data = response.json()

def process_streams(data):
    rows = []
    for stream in data['streams']:
        row = {
            "Province": stream["province"],
            "Category / Stream": stream["name"],
            "Program Status": stream["status"],
            "Express Entry-Linked": "Yes" if stream["express_entry"] == 1 else "No",
            "Job Required": "Yes" if stream["job_offer_required"] == 1 else "No",
            "Stream Link": stream["url"] if stream["url"] else "N/A"
        }
        rows.append(row)
    return pd.DataFrame(rows)

df = process_streams(data)
print(df)
programs=df.to_dict(orient='records')
pd.DataFrame(programs).to_csv('all_programs.csv')

   Province         Category / Stream          Program Status  \
0   Alberta  Accelerated Tech Pathway  Accepting applications   
1   Alberta        Rural Entrepreneur             Coming soon   
2   Alberta             Rural Renewal             Coming soon   
3   Alberta             Express Entry                Passive*   
4   Alberta       Alberta Opportunity  Accepting applications   
..      ...                       ...                     ...   
93    Yukon   Yukon Community Program  Accepting applications   
94    Yukon             Express Entry  Accepting applications   
95    Yukon            Skilled Worker  Accepting applications   
96    Yukon    Critical Impact Worker  Accepting applications   
97    Yukon          Business Nominee  Accepting applications   

   Express Entry-Linked Job Required  \
0                   Yes          Yes   
1                    No           No   
2                    No          Yes   
3                   Yes           No   
4                  

In [6]:
from openai import OpenAI
import os
# from dotenv import load_dotenv

# load_dotenv()

api_key = 'sk-proj-MLqWQdMAx3lgry5b0ceXT3BlbkFJacTRA7ZQY85esF1S5KAF'
# api_key = os.getenv('API_KEY')
client = OpenAI(
    api_key = api_key,
)


In [7]:
from urllib.parse import urlparse
def uri_validator(x):
    try:
        result = urlparse(x)
        return all([result.scheme, result.netloc])
    except AttributeError:
        return False

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('API_KEY')
client = OpenAI(
    api_key = api_key,
)

In [8]:
from selenium.common.exceptions import WebDriverException
import time
import html2text

def get_program_info(index, driver, program):
    url = program['Stream Link']
    province = program['Province']

    if not uri_validator(url):
        return []

    retries = 5
    delay = 2
    soup = None

    for i in range(retries):
        try:
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            break
        except WebDriverException as e:
            if i == retries - 1:
                print(f"Failed to retrieve page after {retries} attempts: {e}")
                return []
            time.sleep(delay)
            delay *= 2

    if soup is None:
        return []


    article = soup.find('article')
    if not article:
        print("Article not found in HTML")
        return []

    size = len(article.text.split())
    if size > 16000:
        try:
            article = html2text.html2text(article)
        except:
            print(url + 'a lot ...')

    prompt = (
        "Extract the following information from the provided HTML content and make sure you give them as plain text, not markdown:\n"
        "1. Stream_name\n"
        "2. Substream_name (name of the sub-categories as specified in the stream)\n"
        "3. Education (type and note, type can be degrees such as elementary, secondary, and post secondary)\n"
        "4. Work_experience (job category name, NOC ID, note, and work experience length)\n"
        "5. Language_proficiency (level requirement and note)\n"
        "6. Province\n"
        "7. Last_updated\n\n"
        "If any specific information is not found, it should be labeled as Not Specified.\n\n"
        "Format the output as a JSON array with objects containing the above fields. Ensure the JSON is properly formatted and each object is correctly"
        "closed with curly braces. The array should also be enclosed with square brackets.\n\n"
        "Examples:\n\n"
        "Example 1:\n"
        "{\n"
        "  \"Stream_name\": \"Express Entry\",\n"
        "  \"Substream_name\": \"Skilled Worker\",\n"
        "  \"Education\": {\n"
        "    \"Education_type\": \"Post secondary\",\n"
        "    \"Education_note\": \"The Skilled Worker category is for international skilled workers who have post-secondary education or training.\"\n"
        "  },\n"
        "  \"Work_experience\": {\n"
        "    \"Job_category_name\": \"Professional, management, technical, trade or other skilled occupation\",\n"
        "    \"Job_NOC_id\": \"NOC 0, A, or B\",\n"
        "    \"Job_note\": \"Candidates must have a full-time permanent qualifying job offer in a skilled occupation from a B.C. employer.\",\n"
        "    \"Work_experience_length\": \"2 years\"\n"
        "  },\n"
        "  \"Language_proficiency\": {\n"
        "    \"Language_level_requirement\": \"CLB 7 or higher\",\n"
        "    \"Language_proficiency_note\": \"Candidates must meet a mandatory language proficiency level.\"\n"
        "  }\n"
        "}\n\n"
        "Example 2:\n"
        "{\n"
        "  \"Stream_name\": \"Express Entry\",\n"
        "  \"Substream_name\": \"International Graduate\",\n"
        "  \"Education\": {\n"
        "    \"Education_type\": \"secondary\",\n"
        "    \"Education_note\": \"Candidates must have graduated from a Canadian highschool or college in the last three years.\"\n"
        "  },\n"
        "  \"Work_experience\": {\n"
        "    \"Job_category_name\": \"management, statistics \",\n"
        "    \"Job_NOC_id\": \"NOC 0, 2,3\",\n"
        "    \"Job_note\": \"Candidates must have a full-time permanent job offer from a B.C. employer.\",\n"
        "    \"Work_experience_length\": \"1 year\"\n"
        "  },\n"
        "  \"Language_proficiency\": {\n"
        "    \"Language_level_requirement\": \"IELTS 5 or higher\",\n"
        "    \"Language_proficiency_note\": \"Candidates must meet a mandatory language proficiency level.\"\n"
        "  }\n"
        "}\n\n"
        "program:\n"
        f"{program}\n"
        "HTML content:\n"
        f"{article.text}\n"
    )

    print("\n--------------------------")
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
    except:
        print("failure")
        return []

    return response.choices[0].message.content


In [9]:
import numpy as np
def is_safe(x):
    try:
        if x in [None, ''] or pd.isna(x):
            return False
    except:
        return True
    return True

def json_loads(x):

     try:
        result=json.loads(x)
     except  Exception as e:
        result=x
        print(e)
     return result

def safe_json_loads(x):
    if not is_safe(x):
        return []

    try:
        result=json.loads(x)
    except Exception as e:
       result=json_loads(str(x).replace("'","\"").replace("None","null").replace("\"s","'s"))
    return result

In [10]:
data = pd.DataFrame()
strings = []
failed = []
for program in programs:
    json_string=get_program_info(0, driver, program)
    # print(json_string)
    if len(json_string):
        # print(list(json_string))
        try:
            # py_object = json.loads(json_string)
            json_string = json_string.replace("json\n", "").replace("`", "").strip()
            new_data = safe_json_loads(json_string)
            if not len(new_data):
                continue

            print(new_data)
            try:
                new_df = pd.DataFrame(new_data)
            except:
                failed.append(program)
                print("Failed!")
            new_df.dropna(how='all', inplace=True)
            new_df['stream'] = program['Category / Stream']
            new_df['link'] = program['Stream Link']
            data = pd.concat([data, new_df], ignore_index=True)
            print(data)
            print("JSON loaded successfully!")
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e.msg}")
            print(f"Error at line {e.lineno}, column {e.colno}, char {e.pos}")
    else:
        strings.append(program)
data

Streaming output truncated to the last 5000 lines.

                                      Work_experience  \
0   {'Job_category_name': 'Not Specified', 'Job_NO...   
1   {'Job_category_name': 'Technology worker in a ...   
2   {'Job_category_name': 'Not Specified', 'Job_NO...   
3   {'Job_category_name': 'Not Specified', 'Job_NO...   
4   {'Job_category_name': 'Professional, managemen...   
5   {'Job_category_name': 'Not Specified', 'Job_NO...   
6   {'Job_category_name': 'Professional, managemen...   
7   {'Job_category_name': 'Not Specified', 'Job_NO...   
8   {'Job_category_name': 'Professional, managemen...   
9   {'Job_category_name': 'management, statistics ...   
10  {'Job_category_name': 'Professional, managemen...   
11  {'Job_category_name': 'Management, statistics'...   
12  {'Job_category_name': 'Professional, managemen...   
13  {'Job_category_name': 'Physician, Specialist, ...   
14  {'Job_category_name': 'NOC level 0, A, or B oc...   
15  {'Job_category_name': 'Professio

,Stream_name,Substream_name,Education,Work_experience,Language_proficiency,Province,Last_updated,stream,link
0,Express Entry,General Express Entry Stream,"{'Education_type': 'Not Specified', 'Education...","{'Job_category_name': 'Not Specified', 'Job_NO...",{'Language_level_requirement': 'Not Specified'...,Alberta,14 March 2023,Express Entry,https://www.canadavisa.com/alberta-express-ent...
1,Express Entry,Accelerated Tech Pathway,"{'Education_type': 'Not Specified', 'Education...",{'Job_category_name': 'Technology worker in a ...,{'Language_level_requirement': 'Not Specified'...,Alberta,14 March 2023,Express Entry,https://www.canadavisa.com/alberta-express-ent...
2,Alberta Opportunity,Not Specified,"{'Education_type': 'Post secondary', 'Educatio...","{'Job_category_name': 'Not Specified', 'Job_NO...",{'Language_level_requirement': 'CLB 4 or highe...,Alberta,14 March 2023,Alberta Opportunity,https://www.canadavisa.com/alberta-opportunity...
3,AAIP Farm Stream,Not Specified,"{'Education_type': 'Not Specified', 'Education...","{'Job_category_name': 'Not Specified', 'Job_NO...",{'Language_level_requirement': 'Not Specified'...,Alberta,14 March 2023,Farm,https://www.canadavisa.com/alberta-self-employ...
4,Express Entry,Skilled Worker,"{'Education_type': 'Post secondary', 'Educatio...","{'Job_category_name': 'Professional, managemen...",{'Language_level_requirement': 'Not Specified'...,British Columbia,Not Specified,International Graduate: EEBC Option,https://www.canadavisa.com/express-entry-briti...
...,...,...,...,...,...,...,...,...,...
107,SINP,Farm Owners and Operators,"{'Education_type': 'Not Specified', 'Education...","{'Job_category_name': 'Farming operations', 'J...",{'Language_level_requirement': 'Not Specified'...,Saskatchewan,Not Specified,Yukon Community Program,https://www.canadavisa.com/yukon-provincial-no...
108,SINP,Farm Owners and Operators,"{'Education_type': 'Not Specified', 'Education...","{'Job_category_name': 'Farming operations', 'J...",{'Language_level_requirement': 'Not Specified'...,Saskatchewan,Not Specified,Express Entry,https://www.canadavisa.com/yukon-provincial-no...
109,Yukon,Skilled Worker,"{'Education_type': 'Not Specified', 'Education...","{'Job_category_name': 'Not Specified', 'Job_NO...",{'Language_level_requirement': 'Not Specified'...,Yukon,Not Specified,Skilled Worker,https://www.canadavisa.com/ynp-skilled-worker-...
110,Critical Impact Worker,Not Specified,"{'Education_type': 'secondary', 'Education_not...",{'Job_category_name': 'NOC Skill Level C or D ...,{'Language_level_requirement': 'IELTS 4 in eac...,Yukon,Not Specified,Critical Impact Worker,https://www.canadavisa.com/ynp-critical-impact...


In [11]:
print("Number of programs failed:", len(failed))
data.to_csv('results.csv')

Number of programs failed: 6


In [57]:
def manual_json_parse(json_str):
    try:
        # Remove curly braces and split the string into comma-separated segments
        json_str = json_str.strip('{}')
        segments = json_str.split(', ')
        # Extract key-value pairs
        data_dict = {}
        for segment in segments:
            key, value = map(lambda x: x.strip("'"), segment.split(': '))
            data_dict[key] = value
        return data_dict
    except Exception as e:
        return {'Error': 'Failed to parse', 'Original': json_str}

# Apply the parsing function to specific columns
def parse_json_columns(data, columns):
    for column in columns:
        data[column + '_parsed'] = data[column].apply(manual_json_parse)
        new_df = pd.json_normalize(data[column + '_parsed'])
        data = pd.concat([data.drop(columns=[column]), new_df], axis=1)
    return data

# Main function to load, process, and save the CSV data
def process_data(data, output_file_path, columns_to_parse):
    data_processed = parse_json_columns(data, columns_to_parse)
    data_processed.drop(columns = ['Error', 'Original.Job_NOC_ID'], inplace = True)
    data_processed.rename(columns={"Original.Education_type": "education_type",
                                   "Original.Education_note": "education_note",
                                   "Original.Job_category_name": "job_category_name",
                                   "Original.Job_NOC_id": "job_NOC_id",
                                   "Original.Language_level_requirement": "language_level_requirement",
                                   "Original.Language_proficiency_note": "language_proficiency_note"}
                          , inplace = True)
    data_processed.to_csv(output_file_path, index=False)
    print(f"Processed data saved to {output_file_path}")

# Specify the file paths and columns to parse
output_file_path = 'final_output.csv'
columns_to_parse = ['Education', 'Work_experience', 'Language_proficiency']


In [56]:
process_data(data, output_file_path, columns_to_parse)

Processed data saved to path_to_output_file.csv
